<a href="https://colab.research.google.com/github/khanhquynh25/DemoGit/blob/master/AII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# ================== 1. ĐỌC DỮ LIỆU ==================
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/machinevision/bodyfat.csv')

X = data.drop("BodyFat", axis=1).values
y = data["BodyFat"].values
feature_names = data.drop("BodyFat", axis=1).columns.tolist()

# ================== 2. CHIA DATA ==================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ================== 3. XÂY DỰNG ANN ==================
model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# ================== 4. HUẤN LUYỆN ==================
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    verbose=0
)

# ================== 5. GIAO DIỆN TRONG COLAB ==================
# Lấy hàng mặc định đầu tiên
sample_row = data.iloc[0]

# Các ô nhập dữ liệu
inputs = {f: widgets.FloatText(description=f, value=float(sample_row[f])) for f in feature_names}

# Ô nhập BodyFat thực tế
true_value_input = widgets.FloatText(description="BodyFat thực tế", value=float(sample_row["BodyFat"]))

# Nút dự đoán & random
predict_button = widgets.Button(description="🔮 Dự đoán Body Fat", button_style="info")
random_button = widgets.Button(description="🎲 Random Sample", button_style="warning")

# Khu vực hiển thị kết quả
output = widgets.Output()

# Lưu kết quả
results = []

def on_predict_clicked(b):
    with output:
        output.clear_output()
        values = [inputs[f].value for f in feature_names]
        sample = np.array([values])
        sample_scaled = scaler.transform(sample)
        pred = model.predict(sample_scaled)[0][0]

        print(f"👉 Dự đoán % Body Fat: {pred:.2f}")

        true_val = true_value_input.value
        if true_val > 0:
            error = abs(pred - true_val)
            accuracy = max(0, 100 - (error / true_val * 100))  # tránh âm
            print(f"📌 Giá trị thực tế: {true_val:.2f}")
            print(f"📊 Sai số tuyệt đối: {error:.2f}")
            print(f"✅ Độ chính xác dự đoán: {accuracy:.2f}%")

            results.append({
                **{f: val for f, val in zip(feature_names, values)},
                "Predicted": pred,
                "Actual": true_val,
                "Error": error,
                "Accuracy (%)": accuracy
            })

            df_results = pd.DataFrame(results)
            output_path = "/content/drive/MyDrive/machinevision/predictions.xlsx"
            df_results.to_excel(output_path, index=False)
            print(f"\n📂 Kết quả đã lưu vào: {output_path}")
            display(df_results.tail())

def on_random_clicked(b):
    rand_row = data.sample(1).iloc[0]
    for f in feature_names:
        inputs[f].value = float(rand_row[f])
    true_value_input.value = float(rand_row["BodyFat"])
    with output:
        output.clear_output()
        print("🎲 Đã chọn một mẫu ngẫu nhiên từ dataset!")

predict_button.on_click(on_predict_clicked)
random_button.on_click(on_random_clicked)

# Hiển thị form giao diện
form_items = list(inputs.values()) + [true_value_input, widgets.HBox([predict_button, random_button])]
display(*form_items, output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FloatText(value=23.0, description='Age')

FloatText(value=160.0, description='Weight')

FloatText(value=68.0, description='Height')

FloatText(value=37.0, description='Neck')

...

FloatText(value=32.0, description='Biceps')

FloatText(value=27.0, description='Forearm')

FloatText(value=18.0, description='Wrist')

FloatText(value=15.0, description='BodyFat thực tế')

Output()

In [ ]:
def on_predict_clicked(b):
    with output:
        output.clear_output()
        values = [inputs[f].value for f in feature_names]
        sample = np.array([values])
        sample_scaled = scaler.transform(sample)
        pred = model.predict(sample_scaled)[0][0]

        print(f"👉 Dự đoán % Body Fat: {pred:.2f}")

        true_val = true_value_input.value
        if true_val > 0:
            error = abs(pred - true_val)
            accuracy = max(0, 100 - (error / true_val * 100))  # tránh âm
            print(f"📌 Giá trị thực tế: {true_val:.2f}")
            print(f"📊 Sai số tuyệt đối: {error:.2f}")
            print(f"✅ Độ chính xác dự đoán: {accuracy:.2f}%")

            # ✅ Chỉ lưu kết quả nếu độ chính xác >= 80%
            if accuracy >= 80:
                results.append({
                    **{f: val for f, val in zip(feature_names, values)},
                    "Predicted": pred,
                    "Actual": true_val,
                    "Error": error,
                    "Accuracy (%)": accuracy
                })

                df_results = pd.DataFrame(results)
                output_path = "/content/drive/MyDrive/machinevision/predictions.xlsx"
                df_results.to_excel(output_path, index=False)
                print(f"\n📂 Kết quả (accuracy ≥ 80%) đã lưu vào: {output_path}")
                display(df_results.tail())
            else:
                print("\n⚠️ Độ chính xác < 80% nên không lưu vào Excel!")


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# ================== 1. ĐỌC DỮ LIỆU ==================
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/machinevision/bodyfat.csv')

X = data.drop("BodyFat", axis=1).values
y = data["BodyFat"].values
feature_names = data.drop("BodyFat", axis=1).columns.tolist()

# ================== 2. CHIA DATA ==================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ================== 3. XÂY DỰNG ANN ==================
model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# ================== 4. HUẤN LUYỆN ==================
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    verbose=0
)

# ================== 5. GIAO DIỆN TRONG COLAB ==================
# Lấy hàng mặc định đầu tiên
sample_row = data.iloc[0]

# Các ô nhập dữ liệu
inputs = {f: widgets.FloatText(description=f, value=float(sample_row[f])) for f in feature_names}

# Ô nhập BodyFat thực tế
true_value_input = widgets.FloatText(description="BodyFat thực tế", value=float(sample_row["BodyFat"]))

# Nút dự đoán & random
predict_button = widgets.Button(description="🔮 Dự đoán Body Fat", button_style="info")
random_button = widgets.Button(description="🎲 Random Sample", button_style="warning")

# Khu vực hiển thị kết quả
output = widgets.Output()

# Lưu kết quả
results = []

def on_predict_clicked(b):
    with output:
        output.clear_output()
        values = [inputs[f].value for f in feature_names]
        sample = np.array([values])
        sample_scaled = scaler.transform(sample)
        pred = model.predict(sample_scaled)[0][0]

        print(f"👉 Dự đoán % Body Fat: {pred:.2f}")

        true_val = true_value_input.value
        if true_val > 0:
            error = abs(pred - true_val)
            accuracy = max(0, 100 - (error / true_val * 100))  # tránh âm
            print(f"📌 Giá trị thực tế: {true_val:.2f}")
            print(f"📊 Sai số tuyệt đối: {error:.2f}")
            print(f"✅ Độ chính xác dự đoán: {accuracy:.2f}%")

            results.append({
                **{f: val for f, val in zip(feature_names, values)},
                "Predicted": pred,
                "Actual": true_val,
                "Error": error,
                "Accuracy (%)": accuracy
            })

            df_results = pd.DataFrame(results)
            output_path = "/content/drive/MyDrive/machinevision/predictions.xlsx"
            df_results.to_excel(output_path, index=False)
            print(f"\n📂 Kết quả đã lưu vào: {output_path}")
            display(df_results.tail())

def on_random_clicked(b):
    rand_row = data.sample(1).iloc[0]
    for f in feature_names:
        inputs[f].value = float(rand_row[f])
    true_value_input.value = float(rand_row["BodyFat"])
    with output:
        output.clear_output()
        print("🎲 Đã chọn một mẫu ngẫu nhiên từ dataset!")

predict_button.on_click(on_predict_clicked)
random_button.on_click(on_random_clicked)

# Hiển thị form giao diện
form_items = list(inputs.values()) + [true_value_input, widgets.HBox([predict_button, random_button])]
display(*form_items, output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FloatText(value=23.0, description='Age')

FloatText(value=160.0, description='Weight')

FloatText(value=68.0, description='Height')

FloatText(value=37.0, description='Neck')

...

FloatText(value=32.0, description='Biceps')

FloatText(value=27.0, description='Forearm')

FloatText(value=18.0, description='Wrist')

FloatText(value=15.0, description='BodyFat thực tế')

Output()